<a href="https://colab.research.google.com/github/Spica08/deep-learning-from-scratch-5/blob/main/step8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# set up
import os
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# step8 拡散モデルの理論

## 8.1 VAEから拡散モデルへ
本書での「拡散モデル」は、「Denoising Diffusion Probabilistic Models」を指す。

### 8.1.1 VAEの復習
VAEでは、潜在変数を固定の正規分布よりサンプリングし、潜在変数から観測変数への変換をニューラルネットワークで行うことでデータ生成を行う。また、データxが得られた時の潜在変数zの事後分布を求めるためにもう1つ別のニューラルネットワークを使用する。

### 8.1.2 潜在変数の階層化
これまで見てきたVAEは潜在変数の数が1つだったが、この潜在変数を階層化したモデルを階層型VAEという。  
階層型VAEでは、直前の確率変数だけから決定される。この性質をマルコフ性といい、これを仮定することでパラメータの増加を防ぐ事ができる。そして潜在変数を階層化することでより複雑な表現を可能にする。  
例えば階層をT層にすることを考えると、それぞれの階層でencoder, decoderが必要となり合計2T個のニューラルネットワークが必要になる。このままではTが大きくなった時に実現が困難になってしまう。これを解決するために進化したものが拡散モデルである。

### 8.1.3 拡散モデルへ
階層型VAEは以下の2点を変更するだけで「拡散モデル」となる。  
1. 観測変数と潜在変数の次元を同じにする。  
2. エンコーダは、固定の正規分布によるノイズを追加する  

これに伴って、潜在変数の記号は全てxで統一し、エンコーダのパラメータが不要となる。そして、拡散モデルでは、ノイズを除去する過程をニューラルネットワークでモデル化する。

## 8.2 拡散過程と逆拡散過程
拡散モデルには、1:ノイズを追加する**拡散過程**と、2:ノイズを除去する**逆拡散過程**が存在する。

### 8.2.1 拡散過程
拡散過程では、1つ前の時刻のデータに対してノイズを加える。このノイズの加え方に関して満たすべき条件は、「最終時刻における潜在変数$x_T$が完全なノイズになること」である。  

ここで、$x_T$が完全なノイズ$\mathcal{N}(x_T;0,I)$に従うように、各時刻で加えるノイズの大きさを考える。1つに以下の方法がある。  
\begin{equation}
q(x_T | x_{T - 1}) = \mathcal{N}(x_T;\sqrt{1 - \beta_t}x_{T - 1}, \beta_tI)
\end{equation}
tは各時刻を表し、$\beta_t$はあらかじめ設定する0.01などの値とする。$\beta_t$が大きいほど分散と共に加えるノイズが大きくなる。この値を時刻ごとにT個の値として設定する。この時Tを1000くらいにある程度大きくし各βを適切に設定(ノイズスケジューリングという)すれば、最終的なデータは$\mathcal{N}(x_T;0,I)$に従う(後述)。

\begin{equation}
q(x_T | x_{T - 1}) = \mathcal{N}(x_T;\sqrt{1 - \beta_t}x_{T - 1}, \beta_tI)
\end{equation}
この式は、平均ベクトルが$\sqrt{1 - \beta_t}x_{T - 1}$、共分散行列が$\beta_tI$の正規分布であり、サンプリングされる値は変数変換トリックを用いると

\begin{align}
\epsilon &\backsim \mathcal{N}(\epsilon;0,I)\\
x_T &= \sqrt{1 - \beta_t}x_{T - 1} + \sqrt{\beta_T}\epsilon
\end{align}
と表す事ができる。これは、標準正規分布より$\epsilon$をサンプリングし、$\sqrt{\beta_T}$倍した値をノイズとして加えることを表す。つまり、前時刻のデータをややスケールダウンさせ、小さなノイズを加える、ということを繰り返す。